In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
%cd "/content/drive/Othercomputers/내 노트북/Forked_Models/승객 승하차 분류 유효성/Model 추가 학습/데이터&추가학습된 모델&유틸"

from __future__ import print_function
from __future__ import absolute_import

######################################
# Import Library
######################################
import tensorflow as tf
from tensorflow import keras
from datetime import datetime
import os
from pytz import timezone
import numpy as np
from utils.dev_env_info import getDevEnvInfo
from utils.data import GetOnOffDataset
from glob import glob
from sklearn.metrics import classification_report, confusion_matrix

######################################
# Set GPU
######################################
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = '0'


######################################
# Set GPU Memory
######################################
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
# if gpus:
#   try:
#     tf.config.experimental.set_virtual_device_configuration(
#         gpus[0],
#         [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=10240)])
#   except RuntimeError as e:
#     print(e)


######################################
# Set Functions
######################################
def getConfusionMatrixLog(confusion_matrix_array, class_name):
    dst = ''
    dst += '{0:15s}{1:15s}{2:s}\n'.format('', '', 'Predicted Class')
    for idx in range(len(class_name) + 1):
        if idx == 0:
            dst += '{0:15s}{1:15s}'.format('', '')
            for name in class_name:
                dst += '{0:15s}'.format(name)

        else:
            if idx == 1:
                dst += '\n{0:15s}'.format('True Class')
                dst += '{0:15s}'.format(class_name[idx - 1])

            else:
                dst += '{0:15s}{1:15s}'.format('', class_name[idx - 1])

            for value in confusion_matrix_array[idx - 1]:
                dst += '{0:15s}'.format(str(value))
            dst += '\n'

    return dst


def getClassReportLog(class_report_dict, class_name):
    dst = ''
    dst += '{0:15s}{1:15s}{2:15s}{3:15s}\n'.format('', 'Precision', 'Recall', 'F1-score')
    for name in class_name:
        precision = class_report_dict[name]['precision']
        precision = '{0:.2f}'.format(precision)
        recall = class_report_dict[name]['recall']
        recall = '{0:.2f}'.format(recall)
        f1_score = class_report_dict[name]['f1-score']
        f1_score = '{0:.2f}'.format(f1_score)
        dst += '{0:15s}{1:15s}{2:15s}{3:15s}\n'.format(name,
                                                       precision,
                                                       recall,
                                                       f1_score)

    precision = class_report_dict['weighted avg']['precision']
    precision = '{0:.2f}'.format(precision)
    recall = class_report_dict['weighted avg']['recall']
    recall = '{0:.2f}'.format(recall)
    f1_score = class_report_dict['weighted avg']['f1-score']
    f1_score = '{0:.2f}'.format(f1_score)
    dst += '\n{0:15s}{1:15s}{2:15s}{3:15s}\n'.format('Total',
                                                     precision,
                                                     recall,
                                                     f1_score)

    return dst


######################################
# Global Parameters
######################################
fmt = '%Y-%m-%d %H:%M:%S %Z%z'
cur_dir = os.getcwd()
dataset_dir = os.path.join(cur_dir, "dataset")
model_save_dir = os.path.join(cur_dir, '')
f = open("/content/drive/Othercomputers/내 노트북/Forked_Models/승객 승하차 분류 유효성/Model 추가 학습/데이터&추가학습된 모델&유틸/output/evaluate_log.txt", 'w')
class_name = ['Get On', 'Get Off', 'Idle']


######################################
# Get Development Environment Info
######################################
cur_time = datetime.now(timezone('Asia/Seoul')).strftime(fmt)
timestamp_str = '{0:s}'.format('[Timestamp: ' + cur_time + ']')
tmp_str = timestamp_str + ' 개발 환경 정보'
log_str = tmp_str + '\n'
f.write(log_str)
print(tmp_str)

tmp_str = getDevEnvInfo()
log_str = tmp_str + '\n'
f.write(log_str)
print(tmp_str)


/content/drive/Othercomputers/내 노트북/Forked_Models/승객 승하차 분류 유효성/Model 추가 학습/데이터&추가학습된 모델&유틸
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
[Timestamp: 2023-05-06 22:20:20 KST+0900] 개발 환경 정보
OS:            Ubuntu 20.04.5 LTS 
CPU:           Intel(R) Xeon(R) CPU @ 2.30GHz
GPU_0:         Tesla V100-SXM2-16GB
Memory:        26690604 kB
Storage:       180.00 GiB
TensorFlow:    2.12.0
Keras:         2.12.0
Python:        3.10.11



In [4]:
%cd "/content/drive/Othercomputers/내 노트북/Forked_Models/승객 승하차 분류 유효성/Model 추가 학습"

loaded_model = keras.models.load_model("fs_lstm_get_on_off_chC.h5")

/content/drive/Othercomputers/내 노트북/Forked_Models/승객 승하차 분류 유효성/Model 추가 학습


In [5]:
loaded_model.summary()

Model: "fs_lstm_get_on_off"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_tensor (InputLayer)   [(None, 3, 8)]            0         
                                                                 
 lstm_0 (LSTM)               (None, 3, 8)              544       
                                                                 
 batch_normalization_0 (Batc  (None, 3, 8)             32        
 hNormalization)                                                 
                                                                 
 lstm_1 (LSTM)               (None, 3, 8)              544       
                                                                 
 batch_normalization_1 (Batc  (None, 3, 8)             32        
 hNormalization)                                                 
                                                                 
 lstm_2 (LSTM)               (None, 3, 8)       

In [5]:
######################################
# Get Train Data
######################################
# Start make train numpy array
cur_time = datetime.now(timezone('Asia/Seoul')).strftime(fmt)
timestamp_str = '{0:s}'.format('[Timestamp: ' + cur_time + ']')
tmp_str = timestamp_str + ' Train 데이터 정보 가져오기 시작'
log_str = tmp_str + '\n'
f.write(log_str)
print(tmp_str)

get_on_off_dataset = GetOnOffDataset(dataset_dir)
train_list = get_on_off_dataset.train_list

# Get train data list
cur_time = datetime.now(timezone('Asia/Seoul')).strftime(fmt)
timestamp_str = '{0:s}'.format('[Timestamp: ' + cur_time + ']')
tmp_str = timestamp_str + ' Train 데이터 목록'
log_str = tmp_str + '\n'
f.write(log_str)
print(tmp_str)

tmp_str = ''
for idx, train_json_path in enumerate(train_list):
    tmp_str += str(idx + 1) + '. ' + train_json_path + '\n'
    print(train_json_path)
tmp_str += '\n'
f.write(tmp_str)

# Get train numpy array
cur_time = datetime.now(timezone('Asia/Seoul')).strftime(fmt)
timestamp_str = '{0:s}'.format('[Timestamp: ' + cur_time + ']')
tmp_str = timestamp_str + ' Train 데이터 numpy 배열 정보'
log_str = tmp_str + '\n'
f.write(log_str)
print(tmp_str)
train_x, train_y = get_on_off_dataset.getTrainDatasetNumpyArray()
log_str = ''
log_str += 'Train_X Shape: {}\n'.format(train_x.shape)
log_str += 'Train_Y Shape: {}\n'.format(train_y.shape)
log_str += '\n'
f.write(log_str)
print(log_str)

[Timestamp: 2023-05-06 21:58:52 KST+0900] Train 데이터 정보 가져오기 시작


KeyboardInterrupt: ignored

In [ ]:
#np.save('/content/drive/Othercomputers/내 노트북/Forked_Models/승객 승하차 분류 유효성/Model 추가 학습/데이터&추가학습된 모델&유틸/dataset/train_x_save', train_x)
#np.save('/content/drive/Othercomputers/내 노트북/Forked_Models/승객 승하차 분류 유효성/Model 추가 학습/데이터&추가학습된 모델&유틸/dataset/train_y_save', train_y)

In [6]:
train_x = np.load('/content/drive/Othercomputers/내 노트북/Forked_Models/승객 승하차 분류 유효성/Model 추가 학습/데이터&추가학습된 모델&유틸/dataset/train_x_save.npy')
train_y = np.load('/content/drive/Othercomputers/내 노트북/Forked_Models/승객 승하차 분류 유효성/Model 추가 학습/데이터&추가학습된 모델&유틸/dataset/train_y_save.npy')

In [7]:
# Get Train Dataset API
train_dataset = tf.data.Dataset.from_tensor_slices(dict(x=train_x, y=train_y))
train_dataset = train_dataset.shuffle(24407).batch(1)

In [8]:
loaded_model.save_weights('모델 체크포인트/easy_checkpoint')

In [9]:
def train_step(net, example, optimizer):
  """Trains `net` on `example` using `optimizer`."""
  with tf.GradientTape() as tape:
    output = net(example['x'])
    cate_cross = tf.keras.losses.CategoricalCrossentropy()
    loss = cate_cross(example['y'], output, sample_weight=tf.constant([0.8, 0.18, 0.02]))
    print(output)
    print(example['y'])
  variables = net.trainable_variables
  gradients = tape.gradient(loss, variables)
  optimizer.apply_gradients(zip(gradients, variables))
  return loss

In [10]:
def train_and_checkpoint(net, manager):
  ckpt.restore(manager.latest_checkpoint)
  if manager.latest_checkpoint:
    print("Restored from {}".format(manager.latest_checkpoint))
  else:
    print("Initializing from scratch.")

  sum_loss = 0

  for _ in range(250):
    example = next(iterator)
    #print(example)
    loss = train_step(net, example, opt)
    sum_loss = sum_loss + loss
    ckpt.step.assign_add(1)
    #print("loss {:1.9f}".format(sum_loss.numpy()))
    if int(ckpt.step) % 10 == 0:
      save_path = manager.save()
      print("Saved checkpoint for step {}: {}".format(int(ckpt.step), save_path))
      mean_loss = sum_loss / 10
      print("loss {:1.9f}".format(mean_loss.numpy()))
      sum_loss = 0
  
  net.save('/content/drive/Othercomputers/내 노트북/Forked_Models/승객 승하차 분류 유효성/Model 추가 학습/데이터&추가학습된 모델&유틸/추가학습된 모델/ft_model.h5')

#훈련 하기 (체크포인트 복구하고 이어하는 것도 이걸로 가능) -> Try 1

In [22]:
%cd "/content/drive/Othercomputers/내 노트북/Forked_Models/승객 승하차 분류 유효성/Model 추가 학습"
opt = tf.keras.optimizers.Adam(0.01)

iterator = iter(train_dataset)
ckpt = tf.train.Checkpoint(step=tf.Variable(1), optimizer=opt, net=loaded_model, iterator=iterator)
manager = tf.train.CheckpointManager(ckpt, './모델 체크포인트', max_to_keep=3)

/content/drive/Othercomputers/내 노트북/Forked_Models/승객 승하차 분류 유효성/Model 추가 학습


In [ ]:
%cd "/content/drive/Othercomputers/내 노트북/Forked_Models/승객 승하차 분류 유효성/Model 추가 학습"
train_and_checkpoint(loaded_model, manager)

/content/drive/Othercomputers/내 노트북/Forked_Models/승객 승하차 분류 유효성/Model 추가 학습
Restored from ./모델 체크포인트/easy_checkpoint
tf.Tensor([[0.00508978 0.01060752 0.98430276]], shape=(1, 3), dtype=float32)
tf.Tensor([[0. 0. 1.]], shape=(1, 3), dtype=float32)


In [ ]:
%cd "/content/drive/Othercomputers/내 노트북/Forked_Models/승객 승하차 분류 유효성/Model 추가 학습"
for i in range(3):
  train_and_checkpoint(loaded_model, manager)

In [ ]:
%cd "/content/drive/Othercomputers/내 노트북/Forked_Models/승객 승하차 분류 유효성/Model 추가 학습/데이터&추가학습된 모델&유틸"
!python "evaluate.py"

In [ ]:
%cd "/content/drive/Othercomputers/내 노트북/Forked_Models/승객 승하차 분류 유효성/Apply_to_chC"
!python "evaluate.py"

#훈련 하기 -> Try 2


In [11]:
%cd "/content/drive/Othercomputers/내 노트북/Forked_Models/승객 승하차 분류 유효성/Model 추가 학습"

from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint('json_to_getonoff_chC_back.h5',
															save_best_only=True,
															save_weights_only=True,
															monitor='val_loss',
															mode='min',
															verbose=1)


/content/drive/Othercomputers/내 노트북/Forked_Models/승객 승하차 분류 유효성/Model 추가 학습


In [12]:
import datetime
date = datetime.datetime.now()

# string for time 문자열로 바꿔
date =date.strftime("%y년 %m월 %d일 %H시 %m분")
print(type(date))
print(date)

filepath = "/content/drive/Othercomputers/내 노트북/Forked_Models/승객 승하차 분류 유효성/Model 추가 학습/모델 체크포인트2/"

# 에폭과 val_loss 소수4째자리까지
filename = '{epoch:04d}-{val_loss:.4f}.h5'

mcp = ModelCheckpoint(
    monitor="val_loss",
    mode="auto",
    save_best_only=True,
    filepath= (filepath+date+filename),
    verbose=1 
)

<class 'str'>
23년 05월 06일 13시 05분


In [17]:
hist = loaded_model.fit(
	                train_x,
	                train_y,
	                epochs=20,
	                batch_size=100,
	                validation_split=0.2,
	                verbose=1,
	                callbacks = [mcp]
                )

Epoch 1/20
196/196 [==============================] - ETA: 0s - loss: 0.3381 - accuracy: 0.8756
Epoch 1: val_loss improved from 0.31098 to 0.26323, saving model to /content/drive/Othercomputers/내 노트북/Forked_Models/승객 승하차 분류 유효성/Model 추가 학습/모델 체크포인트2/23년 05월 06일 13시 05분0001-0.2632.h5
196/196 [==============================] - 5s 24ms/step - loss: 0.3381 - accuracy: 0.8756 - val_loss: 0.2632 - val_accuracy: 0.9111
Epoch 2/20
194/196 [============================>.] - ETA: 0s - loss: 0.2981 - accuracy: 0.8881
Epoch 2: val_loss improved from 0.26323 to 0.25711, saving model to /content/drive/Othercomputers/내 노트북/Forked_Models/승객 승하차 분류 유효성/Model 추가 학습/모델 체크포인트2/23년 05월 06일 13시 05분0002-0.2571.h5
196/196 [==============================] - 5s 25ms/step - loss: 0.2979 - accuracy: 0.8882 - val_loss: 0.2571 - val_accuracy: 0.9267
Epoch 3/20
196/196 [==============================] - ETA: 0s - loss: 0.2841 - accuracy: 0.8951
Epoch 3: val_loss did not improve from 0.25711
196/196 [================

In [18]:
loaded_model.save('/content/drive/Othercomputers/내 노트북/Forked_Models/승객 승하차 분류 유효성/Model 추가 학습/데이터&추가학습된 모델&유틸/추가학습된 모델/ft_model.h5')

In [2]:
%cd "/content/drive/Othercomputers/내 노트북/Forked_Models/승객 승하차 분류 유효성/Model 추가 학습/데이터&추가학습된 모델&유틸"
!python "evaluate.py"

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.

[Timestamp: 2023-05-07 19:26:48 KST+0900] 1807. /content/drive/Othercomputers/내 노트북/Forked_Models/승객 승하차 분류 유효성/Model 추가 학습/데이터&추가학습된 모델&유틸/dataset/test/idle_21983.json Test 데이터 인풋 결과
정답: Idle
예측: Idle

[Timestamp: 2023-05-07 19:26:48 KST+0900] 1808. /content/drive/Othercomputers/내 노트북/Forked_Models/승객 승하차 분류 유효성/Model 추가 학습/데이터&추가학습된 모델&유틸/dataset/test/idle_21984.json Test 데이터 인풋 결과
정답: Idle
예측: Idle

[Timestamp: 2023-05-07 19:26:49 KST+0900] 1809. /content/drive/Othercomputers/내 노트북/Forked_Models/승객 승하차 분류 유효성/Model 추가 학습/데이터&추가학습된 모델&유틸/dataset/test/idle_21985.json Test 데이터 인풋 결과
정답: Idle
예측: Idle

[Timestamp: 2023-05-07 19:26:49 KST+0900] 1810. /content/drive/Othercomputers/내 노트북/Forked_Models/승객 승하차 분류 유효성/Model 추가 학습/데이터&추가학습된 모델&유틸/dataset/test/idle_21986.json Test 데이터 인풋 결과
정답: Idle
예측: Get Off

[Timestamp: 2023-05-07 19:26:49 KST+0900] 1811. /content/drive/Othercomputers/내 노트북/Forked_Models/승객 승하차 분류 유효성/Model 추가 학습/데이터&추가학습된 모델&유틸/dataset/